In [ ]:
# 测试api

In [ ]:
# 1.配置文件
# 未来读取这个dict，并返回value部分. 具体见 conf/algo_conf.json
# self.optimizer_conf = self.conf["algo_conf"]["optimizer"]
"optimizer":{
    "FTRL": {
        "scope": "Wide",
        "learning_rate": 0.05,
        "learning_rate_power": -0.5,
        "initial_accumulator_value": 0.1,
        "l1_regularization_strength": 2.0,
        "l2_regularization_strength": 7.0,
        "use_locking": False,
    },
    "AdamAsync": {
        "scope": "Global",
        "learning_rate": 0.05,
        "use_locking": False,
        "clip_gradients": 5.0,
    }
}

import tensorflow as tf
from tensorflow.contrib.layers import optimize_loss

# 2.调用部分
class MyEstimator(object):
    def build_graph(self, ):
        logits, labels, weights = -1, -1, -1 # 通过一些辅助方法获取
        loss = self.compute_loss(logits, labels, weights)
        #
        train_op = self.optimize(loss)

    # 3.optimizer()部分
    def optimize(self, loss):
        with tf.name_scope("Optimize"):
            # update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            train_ops = []
            # 两类variables: 一类是配置文件中通过scope特殊指定的，另一类就是其他(记作global或者default scope)
            scope_variables, global_variables = set(), []
            global_optimizer = None

            for name, config in self.optimizer_conf.items():
                # 读取配置文件
                optimizer = self.find_optimizer(name, config) # for 某个tower, name即为 tower_scope_name
                scope = config['scope'] # for 某个totwer, 后面tower的scope name也要与这个配置保持一致

                # global_variables的optimizer_ops的设置分拆了两部分，当前仅记录必要信息
                if scope.lower() == 'global':
                    global_optimizer = (optimizer, config) # for 默认tower
                    continue # global_optimizer的后续配置在循环外
                
                # 1) 设置 scope_variables 的optimizer_ops
                with tf.name_scope(scope):
                    self.logger.info('Set optimizer for scope: %s' % scope)
                    #
                    # 1.1) 获取scope_variables (非global scope的其他scope)
                    variables = tf.trainable_variables(scope)
                    if not variables:
                        self.logger.warn("No variables to optimize in scope: %s" % scope)
                        continue # 当前scope(即tower)无待训练参数，可以直接跳过
                    scope_variables.update(variables)
                    #
                    # 1.2) 设置scope_variables的optimizer_ops
                    cur_opt_scope = self.optimize_scope(loss, variables, optimizer, 
                                                        config.get('clip_gradients', None))
                    train_ops.append(cur_opt_scope)

            # 2) 设置 global_variables 的optimizer_ops
            # scope_variables之外的其他variables, 都视做一个default scope，共享一个默认的optimizer
            with tf.name_scope('Global'):
                self.logger.info('Set optimizer for scope: global scope')
                # 2.1) 获取global_variables (准确说是 other_variables)
                global_variables = [
                    var for var in tf.trainable_variables()
                    if var not in scope_variables
                ]

                if global_variables:
                    if not global_optimizer:
                        raise ValueError("Config Error: no global_optimizer for variables.") # default global 必须配置

                # 2.2 ）设置 global_variables 的optimizer_ops
                optimizer, config = global_optimizer
                cur_opt_scope = self.optimize_scope(loss, global_variables, optimizer, 
                                                    config.get('clip_gradients', None))
                train_ops.append(cur_opt_scope)

            if len(train_ops) > 1:
                train_op = tf.group(*train_ops)
            elif len(train_ops) == 1:
                train_op = train_ops[0]
            else:
                raise ValueError("Config Error: insufficinet scope.")

            return train_op

    def find_optimizer(opt_name, config_kv):
        lr = config_kv.get("learning_rate", 0.0001)
        use_locking = config_kv.get("use_locking", False)

        # 遍历匹配
        if opt_name == "Adagrad":
            opt = tf.train.AdagradOptimizer(
                learning_rate=lr,
                initial_accumulator_value=config_kv.get(
                    "initial_accumulator_value", 0.1
                ),
                use_locking=use_locking
            )
        elif opt_name == "Adam":
            opt = tf.train.AdamOptimizer(
                learning_rate=lr,
                beta1=config_kv.get("beta1", 0.9),
                beta2=config_kv.get("beta2", 0.999),
                epsilon=config_kv.get("epsilon", 1e-8),
                use_locking=use_locking
            )
        elif opt_name == "AdamAsync":
            opt = tf.train.AdamAsyncOptimizer(
                learning_rate=lr,
                beta1=config_kv.get("beta1", 0.9),
                beta2=config_kv.get("beta2", 0.999),
                epsilon=config_kv.get("epsilon", 1e-8),
                use_locking=use_locking
            )
        elif opt_name == "Adadelta":
            opt = tf.train.AdadeltaOptimizer(
                learning_rate=lr,
                rho=config_kv.get("rho", 0.95),
                epsilon=config_kv.get("epsilon", 1e-8),
                use_locking=use_locking
            )
        elif opt_name == "Momentum":
            opt = tf.train.MomentumOptimizer(
                learning_rate=lr,
                momentum=config_kv.get("momentum", 0.0),
                use_nesterov=config_kv.get("use_nesterov", False),
                use_locking=use_locking
            )
        elif opt_name == "Ftrl":
            opt = tf.train.FtrlOptimizer(
                learning_rate=lr,
                learning_rate_power=config_kv.get("learning_rate_power", 0.05),
                initial_accumulator_value=config_kv.get("initial_accumulator_value", 0.1),
                l1_regularization_strength=config_kv.get("l1_regularization_strength", 0.0),
                l2_regularization_strength=config_kv.get("l2_regularization_strength", 0.0),
                l2_shrinkage_regularization_strength=config_kv.get(
                    "l2_shrinkage_regularization_strength", 0.0
                ),
                use_locking=use_locking
            )
        else:
            raise ValueError("Optimizer Error.")
        
        return opt

    def optimize_scope(self, loss, variables, optimizer, clip_gradients=None):
        # 全部待bp的ops
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            return optimize_loss(
                loss=loss,
                global_step=tf.train.get_global_step(),
                learning_rate=None, 
                optimizer=optimizer, 
                clip_gradients=clip_gradients,
                update_ops=update_ops,
                variables=variables,
                summaries=[
                    "learning_rate",
                    "loss",
                    "global_gradient_norm",
                ],
                increment_global_step=False,
            )

        

